In [14]:
import pyodbc

# Configuración de la conexión
server = '192.168.0.20'
database = 'expreso'
username = 'datalitica'
password = '425am953'
port = 1433

# Cadena de conexión
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server},{port};DATABASE={database};UID={username};PWD={password};Encrypt=true;TrustServerCertificate=False'

# Conectar a la base de datos
try:
    connection = pyodbc.connect(connection_string)
    print("Conexión exitosa")
except pyodbc.Error as e:
    print("Error en la conexión:", e)


Error en la conexión: ('08001', "[08001] [Microsoft][ODBC Driver 17 for SQL Server]Invalid value specified for connection string attribute 'Encrypt' (0) (SQLDriverConnect)")


In [11]:
import socket

def check_connection(ip, port):
    try:
        # Crear un socket
        sock = socket.create_connection((ip, port), timeout=5)
        print(f"Conexión exitosa a {ip}:{port}")
        sock.close()
    except (socket.timeout, socket.error) as e:
        print(f"No se pudo conectar a {ip}:{port}. Error: {e}")

# Verificar la conexión a la IP y puerto especificados
check_connection('192.168.0.20', 1433)

Conexión exitosa a 192.168.0.20:1433
